# Benchmarking Our Algorithm against others

In [1]:
import sys
sys.path.append('../../')
from panav.environment.env import Room

from panav.PBS.PBS import PBS
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning, Simple_MILP_Planning

from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 


from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for ompl::msg::LogLevel already registered; second conversion method ignored.


In [2]:
bloating_r = 0.5
vmax = 1.0

#### 100 agent configuration ####
bloating_r = 0.5
vmax = 1.0

TIMEOUT = 100 # timeout

# Algorithms
algs = {  
        "Seq_Tube": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        "Seq_Tube_nonLazy": lambda env: sequential_planning(Tube_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),      
        "Seq_S2M2": lambda env: sequential_planning(Simple_MILP_Planning,env,vmax,bloating_r,TIMEOUT=TIMEOUT,lazy = False),
        "ST-RRT*": lambda env: sequential_ST_RRTStar(env,vmax,bloating_r)
        }

In [8]:
from os import listdir
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]

env_names = [get_env_name(fn) for fn in listdir('./envs')]

print(env_names)

['MultiTunnel', 'Room', 'SingleTunnel', 'Warehouse']


In [11]:
import os
from panav.environment.utils import reduced_agents_env
from panav.checkpoint import get_latest_checkpoint
import signal

def timeout_handler(signum,frame):
    print("Algorithm timeout")
    raise TimeoutError("Timeout")

N_idx = 0
alg_idx = 0
results = []
timeouted = {alg:False for alg in algs}
iteration = 0


for fn in listdir('./envs'):
    with open(f'./envs/{fn}','rb') as fp:
        env_base = pkl.load(fp)
        
    max_n = len(env_base.starts)

    for N in range(1,max_n+1):
        env = reduced_agents_env(env_base,N)
        for name,alg in algs.items():

            if timeouted[name]:
                continue
            
            print("Env:", fn, "Alg:", name, "N agent:",N)

            result = {}
            
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['plan'] = plan
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            
            if t>TIMEOUT:
                timeouted[name] = True
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                print("On time for algorithm", name, "N agent = ", N)


        # Save checkpoint data
        with open("./data/{}".format(fn),'wb') as fp:
            pkl.dump(results,fp)



Env: MultiTunnel.pkl Alg: Seq_Tube N agent: 1
Planning for agent 0/1
On time for algorithm Seq_Tube N agent =  1
Env: MultiTunnel.pkl Alg: Seq_Tube_nonLazy N agent: 1
Planning for agent 0/1
On time for algorithm Seq_Tube_nonLazy N agent =  1
Env: MultiTunnel.pkl Alg: Seq_S2M2 N agent: 1
Planning for agent 0/1
On time for algorithm Seq_S2M2 N agent =  1
Env: MultiTunnel.pkl Alg: ST-RRT* N agent: 1
Planning for agent 0/1
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


On time for algorithmInfo:    SpaceTimeRRT: Created 73 states (32 start + 41 goal)
 ST-RRT* N agent =  1
Env: MultiTunnel.pkl Alg: Seq_Tube N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube N agent =  2
Env: MultiTunnel.pkl Alg: Seq_Tube_nonLazy N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_Tube_nonLazy N agent =  2
Env: MultiTunnel.pkl Alg: Seq_S2M2 N agent: 2
Planning for agent 0/2
Planning for agent 1/2
On time for algorithm Seq_S2M2 N agent =  2
Env: MultiTunnel.pkl Alg: ST-RRT* N agent: 2
Planning for agent 0/2
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 1/2Info:    SpaceTimeRRT: Created 52 states (18 start + 34 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
On time for algorithm ST-RRT* N agent =  2
Info:    SpaceTimeRRT: Created 86 states (37 start + 49 goal)
Env: MultiTunnel.pkl Alg: Seq_Tube N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube N agent =  3
Env: MultiTunnel.pkl Alg: Seq_Tube_nonLazy N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_Tube_nonLazy N agent =  3
Env: MultiTunnel.pkl Alg: Seq_S2M2 N agent: 3
Planning for agent 0/3
Planning for agent 1/3
Planning for agent 2/3
On time for algorithm Seq_S2M2 N agent =  3
Env: MultiTunnel.pkl Alg: ST-RRT* N agent: 3
Planning for agent 0/3
Debug:   SpaceTimeRRT: Planner range detecte

...

Planning for agent 4/6
Info:    SpaceTimeRRT: Created 64 states (36 start + 28 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Info:    SpaceTimeRRT: Increased time bound factor to 4.00
Planning for agent 5/6
Info:    SpaceTimeRRT: Created 516 states (512 start + 4 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
On time for algorithm ST-RRT* N agent =  6
Info:    SpaceTimeRRT: Created 50 states (23 start + 27 goal)
Env: MultiTunnel.pkl Alg: Seq_Tube N agent: 7
Planning for agent 0/7
Planning for agent 1/7
Planning for agent 2/7


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 3/7
Planning for agent 4/7
Planning for agent 5/7
